In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
import random
import seaborn as sns
import traceback
import uuid
from datetime import datetime, timedelta
from decouple import config
from faker import Faker
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# Promotion = ongoing
# Resignation = ongoing
# Turnover = done


class HRAnalyticsModel:
    def __init__(self):
        self.conn = psycopg2.connect(host=config("PSQL_HOST2"),
                                     database=config("PSQL_DB2"),
                                     user=config("PSQL_USER2"),
                                     password=config("PSQL_PASS2"),
                                     port=config("PSQL_PORT2"))

        self.attendance_df = self.load_attendance()
        self.departments_df = self.load_departments()
        self.employees_df = self.load_employees()
        self.performance_df = self.load_performance()
        self.training_df = self.load_training()


    def load_attendance(self):
        query = """
            SELECT attendance_id, employee_id, date, status, hours_worked
            FROM s_attendance;
        """
        
        return pd.read_sql(query, self.conn)
    
    
    def load_departments(self):
        query = """
            SELECT department_id, department_name, location, budget
            FROM s_departments;
        """
        
        return pd.read_sql(query, self.conn)
    
    
    def load_employees(self):
        query = """
            SELECT employee_id, first_name, last_name, department_id, job_title, hire_date, manager_id, status, salary
            FROM s_employees;
        """
        
        return pd.read_sql(query, self.conn)
    
    
    def load_performance(self):
        query = """
            SELECT performance_id, employee_id, review_date, rating, reviewer_id
            FROM s_performance;
        """
        
        return pd.read_sql(query, self.conn)
    
    
    def load_training(self):
        query = """
            SELECT training_id, employee_id, training_name, category, score, completion_date
            FROM s_training;
        """
        
        return pd.read_sql(query, self.conn)

    
    """Analytics"""
    def calculate_turnover_rate(self):
        total_employees = len(self.employees_df)
        departed = len(self.employees_df[self.employees_df['status'] != 'active'])
        turnover_rate = (departed / total_employees) * 100 if total_employees > 0 else 0
        
        return round(turnover_rate, 2)
    
    
    def analyze_department_performance(self):
        merged_df = pd.merge(self.performance_df,
                             self.employees_df,
                             on='employee_id')
        merged_df = pd.merge(merged_df,
                             self.departments_df,
                             on='department_id')
    
        dept_performance = (merged_df
                            .groupby('department_name')['rating']
                            .agg(['mean', 'count', 'std', 'min', 'max'])
                            .round(2))
    
        return dept_performance.reset_index()

    
    def attendance_analysis(self):
        analysis = {
            'total_days_recorded': len(self.attendance_df),
            'present_days': len(self.attendance_df[self.attendance_df['status'] == 'present']),
            'absent_days': len(self.attendance_df[self.attendance_df['status'] == 'absent']),
            'late_days': len(self.attendance_df[self.attendance_df['status'] == 'late']),
            'average_hours_worked': round(self.attendance_df['hours_worked'].mean(), 2),
            'attendance_rate': round((len(self.attendance_df[self.attendance_df['status'] == 'present']) /
                                      len(self.attendance_df)) * 100, 2)
        }
        
        return analysis


    def salary_analysis(self):
        merged_df = pd.merge(self.employees_df,
                             self.departments_df,
                             on='department_id')
    
        salary_stats = (merged_df
                        .groupby('department_name')['salary']
                        .agg([('avg_salary_million', lambda x: round(x.mean() / 1_000_000, 2)),
                              ('median_salary_million', lambda x: round(np.median(x) / 1_000_000, 2)),
                              ('std_salary_million', lambda x: round(x.std() / 1_000_000, 2)),
                              ('min_salary_million', lambda x: round(x.min() / 1_000_000, 2)),
                              ('max_salary_million', lambda x: round(x.max() / 1_000_000, 2)),
                              ('count', 'count')])
                        .reset_index())
    
        return salary_stats

        
    def retention_analysis(self):
        current_date = datetime.now()
    
        df = self.employees_df.copy()
        df['hire_date'] = pd.to_datetime(df['hire_date'])
        df['tenure_years'] = ((current_date - df['hire_date']).dt.days / 365.25).round(2)
    
        merged_df = pd.merge(df,
                             self.departments_df,
                             on='department_id')
    
        retention_stats = (merged_df
                           .groupby('department_name')['tenure_years']
                           .agg([('mean_tenure', lambda x: round(x.mean(), 2)),
                                 ('median_tenure', lambda x: round(np.median(x), 2)),
                                 ('std_tenure', lambda x: round(x.std(), 2)),
                                 ('employee_count', 'count')])
                           .reset_index())
    
        return retention_stats
    
    
    def training_effectiveness(self):
        latest_performance = (self.performance_df
                              .sort_values('review_date')
                              .drop_duplicates('employee_id', keep='last'))
    
        merged_data = pd.merge(self.training_df,
                               latest_performance[['employee_id', 'rating']],
                               on='employee_id')
    
        effectiveness = (merged_data
                         .groupby('training_name')
                         .agg(avg_score=('score', 'mean'),
                              avg_rating=('rating', 'mean'),
                              participants_count=('employee_id', 'count'))
                         .round(2)
                         .reset_index())
    
        return effectiveness


    def generate_comprehensive_report(self):
        # hitung tenure
        df = self.employees_df.copy()
        df['hire_date'] = pd.to_datetime(df['hire_date'])
        current_date = datetime.now()
        df['tenure_years'] = ((current_date - df['hire_date']).dt.days / 365.25).round(2)
    
        # join untuk hitung jumlah departemen unik
        merged_df = pd.merge(df, self.departments_df, on='department_id')
    
        comprehensive_report = {
            'turnover_rate': self.calculate_turnover_rate(),
            'department_performance': self.analyze_department_performance(),
            'attendance_stats': self.attendance_analysis(),
            'salary_analysis': self.salary_analysis(),
            'retention_analysis': self.retention_analysis(),
            'training_effectiveness': self.training_effectiveness(),
            'employee_summary': {
                'total_employees': len(df),
                'active_employees': len(df[df['status'] == 'active']),
                'departments_count': merged_df['department_name'].nunique(),
                'average_salary': f"Rp {df['salary'].mean():,.0f}",
                'average_tenure': f"{df['tenure_years'].mean():.1f} years"
            }
        }
    
        return comprehensive_report

    
    
    """Visualization"""
    def plot_turnover_by_department(self):
        merged = pd.merge(self.employees_df, self.departments_df,
                          on='department_id', how='left')
    
        # Hitung turnover rate: % karyawan tidak aktif
        turnover_by_dept = (merged.groupby('department_name')['status']
                            .apply(lambda x: (x != 'active').mean() * 100)
                            .round(2))
    
        # Plot
        plt.figure(figsize=(10, 6))
        turnover_by_dept.sort_values().plot(kind='bar', color='skyblue')
        plt.title('Turnover Rate by Department')
        plt.xlabel('Department')
        plt.ylabel('Turnover Rate (%)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    
    def plot_salary_distribution(self):
        plt.figure(figsize=(12, 6))

        # Salinan df untuk jaga data asli
        emp_df = self.employees_df.copy()
        emp_df['salary_million'] = emp_df['salary'] / 1_000_000
    
        # Distribusi salary
        plt.subplot(1, 2, 1)
        sns.histplot(emp_df['salary_million'], bins=20, kde=True, color='skyblue')
        plt.title('Salary Distribution (in millions)')
        plt.xlabel('Salary (Million IDR)')
    
        # Rata-rata salary per department
        merged_df = pd.merge(emp_df, self.departments_df, on='department_id', how='left')
        salary_by_dept = (merged_df.groupby('department_name')['salary_million']
                          .mean()
                          .sort_values())
    
        plt.subplot(1, 2, 2)
        salary_by_dept.plot(kind='bar', color='lightgreen')
        plt.title('Average Salary by Department')
        plt.xlabel('Department')
        plt.ylabel('Average Salary (Million IDR)')
        plt.xticks(rotation=45)
    
        plt.tight_layout()
        plt.show()
    
    def plot_performance_trends(self):
        merged_df = pd.merge(self.performance_df, self.employees_df, on='employee_id')
        merged_df = pd.merge(merged_df, self.departments_df, on='department_id')
    
        # Tambah kolom tahun dari review_date
        merged_df['year'] = pd.to_datetime(merged_df['review_date']).dt.year
    
        # Hitung rata-rata rating per tahun per department
        performance_trend = (merged_df.groupby(['year', 'department_name'])['rating']
                             .mean()
                             .reset_index()
                             .pivot(index='year', columns='department_name', values='rating'))
    
        # Plot
        plt.figure(figsize=(12, 6))
        performance_trend.plot(marker='o')
        plt.title('Performance Rating Trend by Department')
        plt.xlabel('Year')
        plt.ylabel('Average Rating')
        plt.legend(title='Department')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    
    def plot_attendance_patterns(self):
        plt.figure(figsize=(15, 10))

        # Attendance by day of week
        self.attendance_df['day_of_week'] = pd.to_datetime(self.attendance_df['date']).dt.day_name()
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    
        plt.subplot(2, 2, 1)
        attendance_by_day = self.attendance_df['day_of_week'].value_counts().reindex(day_order)
        attendance_by_day.plot(kind='bar', color='lightblue')
        plt.title('Attendance by Day of Week')
        plt.xlabel('Day')
        plt.ylabel('Count')
    
        # Attendance status distribution
        plt.subplot(2, 2, 2)
        status_counts = self.attendance_df['status'].value_counts()
        plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%')
        plt.title('Attendance Status Distribution')
    
        # Average hours by department
        plt.subplot(2, 2, 3)
        merged_attendance = pd.merge(self.attendance_df, self.employees_df, on='employee_id')
        merged_attendance = pd.merge(merged_attendance, self.departments_df, on='department_id')
    
        hours_by_dept = (merged_attendance.groupby('department_name')['hours_worked']
                         .mean()
                         .sort_values())
    
        hours_by_dept.plot(kind='bar', color='orange')
        plt.title('Average Hours Worked by Department')
        plt.xlabel('Department')
        plt.ylabel('Average Hours')
        plt.xticks(rotation=45)
    
        plt.tight_layout()
        plt.show()

In [ ]:
# Initialize the model
hr_model = HRAnalyticsModel()

# Generate comprehensive report
report = hr_model.generate_comprehensive_report()

print("=== HR ANALYTICS REPORT ===")
print(f"Total Employees: {report['employee_summary']['total_employees']}")
print(f"Active Employees: {report['employee_summary']['active_employees']}")
print(f"Turnover Rate: {report['turnover_rate']}%")
print(f"\nAverage Salary: {report['employee_summary']['average_salary']}")
print(f"Average Tenure: {report['employee_summary']['average_tenure']}")

print("\n=== DEPARTMENT PERFORMANCE ===")
print(report['department_performance'])

print("\n=== ATTENDANCE STATS ===")
for key, value in report['attendance_stats'].items():
    print(f"{key}: {value}")

# Generate visualizations
hr_model.plot_turnover_by_department()
hr_model.plot_salary_distribution()
hr_model.plot_performance_trends()
hr_model.plot_attendance_patterns()

# Access raw dataframes if needed
print("\nSample of employees data:")
print(hr_model.employees_df.head())
print(f"\nShape of attendance data: {hr_model.attendance_df.shape}")

## 1. Inheritance Structure
#### Call constructor parent
#### Dictionary for saving ML model
```python
class HRPredictiveModel(HRAnalyticsModel):
    def __init__(self):
        super().__init__()
        self.ml_models = {}
```
##### Explanation: A class that inherits all functionality from HRAnalyticsModel and adds machine learning capabilities

## 2. Feature Engineering (prepare_features_for_ml)
```python
def prepare_features_for_ml(self):
    features_df = self.employees_df.copy()
```
#### A. Numerical Feature Engineering:
##### Years worked & Handle missing values
```python
features_df['tenure'] = (datetime.now() - pd.to_datetime(features_df['hire_date'])).dt.days / 365.25
features_df['tenure'] = features_df['tenure'].fillna(0)
```
##### Salary normalization gaji (0-1)
```python
features_df['salary_normalized'] = features_df['salary'] / features_df['salary'].max()
features_df['salary_normalized'] = features_df['salary_normalized'].fillna(0)
```
#### B. Performance Data Aggregation:
##### Aggregate data performance per employee
```python
perf_agg = self.performance_df.groupby('employee_id')['rating'].agg(['mean', 'std', 'count']).reset_index()
perf_agg.columns = ['employee_id', 'performance_mean', 'performance_std', 'performance_count']
```
##### Merge with main data
```python
features_df = features_df.merge(perf_agg, on='employee_id', how='left')
```
##### Handle missing values
```python
features_df['performance_mean'] = features_df['performance_mean'].fillna(features_df['performance_mean'].median())
features_df['performance_std'] = features_df['performance_std'].fillna(0)
features_df['performance_count'] = features_df['performance_count'].fillna(0)
```
#### C. Attendance Data Aggregation:
##### Aggregate data attendance (Average working hours and attendance rate (0-1))
```python
attendance_agg = self.attendance_df.groupby('employee_id').agg({
    'hours_worked': 'mean',
    'status': lambda x: (x == 'absent').mean()
})
```
#### D. Categorical Data Encoding:
##### Convert department ke one-hot encoding
```python
if 'department' in features_df.columns:
    features_df = pd.get_dummies(features_df, columns=['department'], prefix='dept')
```
##### Convert job title ke one-hot encoding
```python
if 'job_title' in features_df.columns:
    features_df = pd.get_dummies(features_df, columns=['job_title'], prefix='job')
```
## 3. Dataset Preparation (create_turnover_prediction_dataset)
##### Get engineered features
```python
def create_turnover_prediction_dataset(self):
    features_df = self.prepare_features_for_ml()
```
#### A. Label Creation:
##### Create target variable (label) with 1 = resigned, 0 = active
```python
y = (features_df['status'] != 'active').astype(int)
print(f"Label value counts: {y.value_counts().to_dict()}")
```
#### B. Feature Selection:
##### Drop columns that are not needed for training

```python
columns_to_drop = ['employee_id', 'first_name', 'last_name', 'email', 'hire_date', 'status']
columns_to_drop = [col for col in columns_to_drop if col in features_df.columns]

X = features_df.drop(columns_to_drop, axis=1, errors='ignore')
```
##### Ensure will_resign doesn't exist (if it does)
```python
if 'will_resign' in X.columns:
    X = X.drop('will_resign', axis=1)
```
## 4. Model Training (train_turnover_prediction_model)
##### Get features and labels
```python
def train_turnover_prediction_model(self):
    X, y = self.create_turnover_prediction_dataset()
```
#### A. Data Preprocessing:
##### Save features name
##### Handle missing values with median imputation
```python
feature_names = X.columns.tolist()

imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
```
#### B. Train-Test Split:
##### Split data 80-20 with stratification
##### stratify=y for Maintain class distribution
```python
X_train, X_test, y_train, y_test = train_test_split(X_imputed,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)
```
#### C. Model Training:
##### Use the Random Forest classifier
##### Training
```python
model = RandomForestClassifier(n_estimators=100,
                               random_state=42,
                               class_weight='balanced')
model.fit(X_train, y_train)
```
#### D. Model Evaluation:
##### Predict dan evaluate
##### Detailing metrics
```python
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
```
#### E. Model Storage:
##### Save models dan preprocessing components
```python
model.feature_names_ = feature_names
self.ml_models['turnover_prediction'] = model
self.ml_models['imputer'] = imputer
self.ml_models['feature_names'] = feature_names
```
## 5. Prediction (predict_turnover_risk)
##### Auto-train if the model doesn't exist yet
```python
def predict_turnover_risk(self, employee_id=None):
    if 'turnover_prediction' not in self.ml_models:
        self.train_turnover_prediction_model()
```
#### A. Feature Preparation for Prediction:
##### Prepare features in the same way as training.
```python
features_df = self.prepare_features_for_ml()
columns_to_drop = ['employee_id', 'first_name', 'last_name', 'email', 'hire_date', 'status']
columns_to_drop = [col for col in columns_to_drop if col in features_df.columns]

X_all = features_df.drop(columns_to_drop, axis=1, errors='ignore')
```
#### B. Feature Alignment:
##### Ensure features match training
##### Handle missing features with loop and zero
##### Handle extra features and discard unnecessary ones
##### Sort features according to training
```python
feature_names = self.ml_models['feature_names']
missing_features = set(feature_names) - set(X_all.columns)

for feature in missing_features:
    X_all[feature] = 0

extra_features = set(X_all.columns) - set(feature_names)
X_all = X_all.drop(list(extra_features), axis=1)
X_all = X_all[feature_names]
```
#### C. Prediction:
##### Preprocessing dan prediction
##### Apply imputer
##### Probability class 1
##### Format results
```python
X_all_imputed = self.ml_models['imputer'].transform(X_all)
predictions = self.ml_models['turnover_prediction'].predict_proba(X_all_imputed)[:, 1]

results['turnover_risk'] = (predictions * 100).round(2)
```
## 6. Feature Importance Analysis
##### Get importance scores from the model & Importance scores from Random Forest
```python
def analyze_feature_importance(self):
    feature_importances = pd.DataFrame({
        'feature': model.feature_names_,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
```
## Engineered Key Features:
- tenure - Years of service
- salary_normalized - Normalized salary (0-1)
- performance_mean - Average performance rating
- performance_std - Performance consistency
- avg_hours_worked - Average working hours
- absence_rate - Absence rate (0-1)
- One-hot encoded departments - dept_IT, dept_HR, etc.
- One-hot encoded job titles - job_Manager, job_Analyst, etc.

## Algorithms Used:
- Random Forest - Robust ensemble method for classification
- Class Weight Balancing - Handle imbalance between active vs resigned
- Median Imputation - Handle missing values
- Stratified Sampling - Maintain class distribution in split

In [67]:
"""Sample ML turnover prediction"""
class HRPredictiveModel(HRAnalyticsModel):
    def __init__(self):
        super().__init__()
        self.ml_models = {}

    def prepare_features_for_ml(self):
        features_df = self.employees_df.copy()

        # Feature engineering - make sure handle missing values
        features_df['tenure'] = (datetime.now() - pd.to_datetime(features_df['hire_date'])).dt.days / 365.25
        features_df['tenure'] = features_df['tenure'].fillna(0)

        features_df['salary_normalized'] = features_df['salary'] / features_df['salary'].max()
        features_df['salary_normalized'] = features_df['salary_normalized'].fillna(0)

        # Aggregate performance data
        perf_agg = self.performance_df.groupby('employee_id')['rating'].agg(['mean', 'std', 'count']).reset_index()
        perf_agg.columns = ['employee_id', 'performance_mean', 'performance_std', 'performance_count']

        features_df = features_df.merge(perf_agg, on='employee_id', how='left')

        # Fill missing performance values
        features_df['performance_mean'] = features_df['performance_mean'].fillna(features_df['performance_mean'].median())
        features_df['performance_std'] = features_df['performance_std'].fillna(0)
        features_df['performance_count'] = features_df['performance_count'].fillna(0)

        # Aggregate attendance data
        if not self.attendance_df.empty:
            attendance_agg = self.attendance_df.groupby('employee_id').agg({
                'hours_worked': 'mean',
                'status': lambda x: (x == 'absent').mean()  # absence rate
            }).reset_index()
            attendance_agg.columns = ['employee_id', 'avg_hours_worked', 'absence_rate']
            features_df = features_df.merge(attendance_agg, on='employee_id', how='left')

            # Fill missing attendance values
            features_df['avg_hours_worked'] = features_df['avg_hours_worked'].fillna(features_df['avg_hours_worked'].median())
            features_df['absence_rate'] = features_df['absence_rate'].fillna(0)

        else:
            features_df['avg_hours_worked'] = 8.0  # default value
            features_df['absence_rate'] = 0.0

        # Convert categorical variables - make sure existing columns
        if 'department' in features_df.columns:
            features_df = pd.get_dummies(features_df, columns=['department'], prefix='dept')

        if 'job_title' in features_df.columns:
            features_df = pd.get_dummies(features_df, columns=['job_title'], prefix='job')

        return features_df

    def create_turnover_prediction_dataset(self):
        features_df = self.prepare_features_for_ml()

        if 'status' not in features_df.columns:
            print("Error: 'status' column not found in features_df")
            return None, None

        y = (features_df['status'] != 'active').astype(int)
        print(f"Label value counts: {y.value_counts().to_dict()}")
        columns_to_drop = ['employee_id', 'first_name', 'last_name', 'email', 'hire_date', 'status']
        columns_to_drop = [col for col in columns_to_drop if col in features_df.columns]
        x = features_df.drop(columns_to_drop, axis=1, errors='ignore')

        if 'will_resign' in x.columns:
            x = x.drop('will_resign', axis=1)

        print(f"X columns: {x.columns.tolist()}")
        print(f"X shape: {x.shape}, y shape: {y.shape}")

        return x, y

    def train_turnover_prediction_model(self):
        try:
            x, y = self.create_turnover_prediction_dataset()
    
            if x is None or y is None:
                print("Failed to create dataset")
                return None
    
            # Simpan nama fitur mentah (urutan asli)
            feature_names_raw = x.columns.tolist()
    
            # Tentukan fitur numerik & kategorikal secara dinamis
            numeric_features = x.select_dtypes(include=['number']).columns.tolist()
            categorical_features = x.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
    
            print(f"Raw feature names: {feature_names_raw}")
            print(f"Numeric features: {numeric_features}")
            print(f"Categorical features: {categorical_features}")
    
            # Buat transformer hanya untuk grup yang tidak kosong
            transformers = []
            if numeric_features:
                numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])
                transformers.append(('num', numeric_transformer, numeric_features))
    
            if categorical_features:
                categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                                          ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
                
                transformers.append(('cat', categorical_transformer, categorical_features))
    
            preprocessor = ColumnTransformer(transformers=transformers, remainder='drop')
    
            # Pipeline utama: preprocessing + classifier
            pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('classifier', RandomForestClassifier(n_estimators=100,
                                                                             random_state=42,
                                                                             class_weight='balanced'))])
    
            # Split (pakai x mentah — pipeline akan meng-handle impute/encode)
            x_train, x_test, y_train, y_test = train_test_split(x,
                                                                y,
                                                                test_size=0.2,
                                                                random_state=42,
                                                                stratify=y)
    
            # Fit pipeline
            pipeline.fit(x_train, y_train)
    
            # Evaluasi
            y_pred = pipeline.predict(x_test)
            print("Model Evaluation:")
            print(classification_report(y_test, y_pred))
    
            # --- Attach atribut yang dibutuhkan oleh cell test ---
            # feature_names_ -> raw feature names (sama seperti X.columns saat training)
            pipeline.feature_names_ = feature_names_raw
            transformed_feature_names = []
            
            if numeric_features:
                transformed_feature_names.extend(numeric_features)
    
            if categorical_features:
                # akses encoder setelah fit
                try:
                    cat_transformer = preprocessor.named_transformers_['cat']
                    # cat_transformer adalah Pipeline; ambil step encoder
                    encoder = cat_transformer.named_steps.get('encoder', cat_transformer)
                    
                    # dapatkan nama fitur hasil encode
                    try:
                        cat_feature_names = encoder.get_feature_names_out(categorical_features)
                        
                    except AttributeError:
                        # fallback untuk sklearn lama
                        cat_feature_names = encoder.get_feature_names(categorical_features)
                        
                    transformed_feature_names.extend(list(cat_feature_names))
                    
                except Exception:
                    # kalau tidak ada fitur kategorikal atau encoder gagal,
                    # biarkan transformed_feature_names seperti numeric saja
                    pass
    
            pipeline.transformed_feature_names_ = transformed_feature_names
    
            # Simpan model & metadata ke self.ml_models
            self.ml_models['turnover_prediction'] = pipeline
            self.ml_models['feature_names'] = feature_names_raw
            self.ml_models['transformed_feature_names'] = transformed_feature_names
    
            # Jika classifier punya feature_importances_, map ke nama fitur ter-transform
            clf = pipeline.named_steps['classifier']
            if hasattr(clf, 'feature_importances_') and len(transformed_feature_names) == len(clf.feature_importances_):
                fi = pd.DataFrame({
                    'feature': transformed_feature_names,
                    'importance': clf.feature_importances_
                }).sort_values('importance', ascending=False).reset_index(drop=True)
                self.ml_models['feature_importances'] = fi
                
            else:
                # Bila ukurannya mismatch (mis. tidak ada fitur kategorikal, dsb.), coba partial mapping atau skip
                self.ml_models['feature_importances'] = None
    
            print(f"Model trained successfully with {len(feature_names_raw)} raw features "
                  f"and {len(transformed_feature_names)} transformed features.")
    
            return pipeline
    
        except Exception as e:
            print(f"Error training model: {e}")
            traceback.print_exc()
            
            return None

    def predict_turnover_risk(self):
        try:
            if 'turnover_prediction' not in self.ml_models:
                print("Training model first...")
                self.train_turnover_prediction_model()
    
                if 'turnover_prediction' not in self.ml_models:
                    return None
    
            features_df = self.prepare_features_for_ml()
    
            # Drop kolom non-feature
            columns_to_drop = ['employee_id', 'first_name', 'last_name', 'email', 'hire_date', 'status']
            columns_to_drop = [col for col in columns_to_drop if col in features_df.columns]
    
            x_all = features_df.drop(columns_to_drop, axis=1, errors='ignore')
    
            if 'will_resign' in x_all.columns:
                x_all = x_all.drop('will_resign', axis=1)
    
            feature_names = self.ml_models['feature_names']
            print(f"Expected features: {feature_names}")
            print(f"Available features: {x_all.columns.tolist()}")
    
            # Sesuaikan kolom dengan training
            missing_features = set(feature_names) - set(x_all.columns)
            extra_features = set(x_all.columns) - set(feature_names)
    
            if missing_features:
                print(f"Warning: Missing features {missing_features}, filling with 0")
                for feature in missing_features:
                    x_all[feature] = 0
    
            if extra_features:
                print(f"Warning: Dropping extra features {extra_features}")
                x_all = x_all.drop(list(extra_features), axis=1)
    
            # Urutkan sesuai training
            x_all = x_all[feature_names]
    
            # langsung pakai pipeline (tidak perlu imputer manual)
            predictions = self.ml_models['turnover_prediction'].predict_proba(x_all)[:, 1]
    
            results = features_df[['employee_id', 'first_name', 'last_name']].copy()
    
            # Add department jika ada
            if 'department' in features_df.columns:
                results['department'] = features_df['department']
                
            elif 'dept_IT' in features_df.columns:  # contoh one-hot
                dept_columns = [col for col in features_df.columns if col.startswith('dept_')]
                
                if dept_columns:
                    results['department'] = features_df[dept_columns].idxmax(axis=1).str.replace('dept_', '')
    
            results['turnover_risk'] = (predictions * 100).round(2)
            results['status'] = features_df['status']
    
            return results.sort_values('turnover_risk', ascending=False)
    
        except Exception as e:
            print(f"Error predicting turnover risk: {e}")
            traceback.print_exc()
            
            return None


    def analyze_feature_importance(self):
        try:
            if 'turnover_prediction' not in self.ml_models:
                self.train_turnover_prediction_model()

            model = self.ml_models['turnover_prediction']
            x, _ = self.create_turnover_prediction_dataset()

            feature_importances = pd.DataFrame({
                'feature': model.feature_names_,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)

            plt.figure(figsize=(12, 8))
            sns.barplot(data=feature_importances.head(10), x='importance', y='feature')
            plt.title('Top 10 Features Affecting Employee Turnover')
            plt.tight_layout()
            plt.show()

            return feature_importances

        except Exception as e:
            print(f"Error analyzing feature importance: {e}")

            return None

In [68]:
# Testing
try:
    hr_ml_model = HRPredictiveModel()
    
    print("=== TRAINING ===")
    ml_model = hr_ml_model.train_turnover_prediction_model()
    
    if ml_model is not None:
        print("=== FEATURE VALIDATION ===")
        print("Model feature names:", ml_model.feature_names_)
        
        # Prepare features untuk prediction
        features_df = hr_ml_model.prepare_features_for_ml()
        columns_to_drop = ['employee_id', 'first_name', 'last_name', 'email', 'hire_date', 'status']
        columns_to_drop = [col for col in columns_to_drop if col in features_df.columns]
        X_pred = features_df.drop(columns_to_drop, axis=1, errors='ignore')
        
        print("Prediction features:", X_pred.columns.tolist())
        print("Feature match:", set(ml_model.feature_names_) == set(X_pred.columns))
        
        print("=== PREDICTION ===")
        risk_scores = hr_ml_model.predict_turnover_risk()
        if risk_scores is not None:
            print("\nTop 10 employees with highest turnover risk:")
            print(risk_scores.head(10))
            
            # Analisis hasil
            print(f"\nRisk score statistics:")
            print(f"Min: {risk_scores['turnover_risk'].min():.2f}%")
            print(f"Max: {risk_scores['turnover_risk'].max():.2f}%")
            print(f"Avg: {risk_scores['turnover_risk'].mean():.2f}%")
            print(f"Active employees avg risk: {risk_scores[risk_scores['status'] == 'active']['turnover_risk'].mean():.2f}%")
            print(f"Resigned employees avg risk: {risk_scores[risk_scores['status'] == 'resigned']['turnover_risk'].mean():.2f}%")
                
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

/var/folders/p3/dg0b0x211vx9yftt50b2n2lh0000gn/T/ipykernel_60294/3808025729.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, self.conn)
/var/folders/p3/dg0b0x211vx9yftt50b2n2lh0000gn/T/ipykernel_60294/3808025729.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, self.conn)
/var/folders/p3/dg0b0x211vx9yftt50b2n2lh0000gn/T/ipykernel_60294/3808025729.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, self.conn)
/var/folders/p3/dg0b0x211vx9yft

=== TRAINING ===
Label value counts: {0: 160, 1: 40}
X columns: ['department_id', 'manager_id', 'salary', 'tenure', 'salary_normalized', 'performance_mean', 'performance_std', 'performance_count', 'avg_hours_worked', 'absence_rate', 'job_Account Manager', 'job_Accountant', 'job_Auditor', 'job_Business Development', 'job_Content Creator', 'job_Data Analyst', 'job_Finance Manager', 'job_Financial Analyst', 'job_HR Manager', 'job_HR Specialist', 'job_IT Manager', 'job_Logistics Coordinator', 'job_Marketing Manager', 'job_Marketing Specialist', 'job_Operations Manager', 'job_Recruiter', 'job_SEO Analyst', 'job_Sales Representative', 'job_Software Engineer', 'job_Supply Chain Specialist', 'job_System Admin', 'job_Training Coordinator']
X shape: (200, 32), y shape: (200,)
Raw feature names: ['department_id', 'manager_id', 'salary', 'tenure', 'salary_normalized', 'performance_mean', 'performance_std', 'performance_count', 'avg_hours_worked', 'absence_rate', 'job_Account Manager', 'job_Account

In [ ]:
class HRPredictiveResignedModel(HRAnalyticsModel):
    def __init__(self):
        super().__init__()
        self.ml_models = {}

    def prepare_features(self):
        # Attendance features
        att_agg = self.attendance_df.groupby("employee_id").agg({
            "hours_worked": "mean",
            "status": lambda x: (x == "Absent").sum() / len(x)
        }).reset_index()
        att_agg.rename(columns={"hours_worked": "avg_hours", "status": "absent_ratio"}, inplace=True)

        # Performance features
        perf_agg = self.performance_df.groupby("employee_id")["rating"].mean().reset_index()
        perf_agg.rename(columns={"rating": "avg_rating"}, inplace=True)

        # Training features
        train_agg = self.training_df.groupby("employee_id")["score"].mean().reset_index()
        train_agg.rename(columns={"score": "avg_training_score"}, inplace=True)

        # Merge to employees
        df = self.employees_df.merge(att_agg, on="employee_id", how="left") \
                              .merge(perf_agg, on="employee_id", how="left") \
                              .merge(train_agg, on="employee_id", how="left") \
                              .merge(self.departments_df, on="department_id", how="left")

        # Tenure
        df["hire_date"] = pd.to_datetime(df["hire_date"])
        df["tenure_days"] = (pd.Timestamp.now() - df["hire_date"]).dt.days

        # Target
        df["resigned"] = df["status"].apply(lambda x: 1 if str(x).lower() == "resigned" else 0)

        # Bersihin NaN
        df.fillna(0, inplace=True)

        X = df[["salary", "avg_hours", "absent_ratio", "avg_rating", 
                "avg_training_score", "tenure_days", "budget"]]
        y = df["resigned"]

        return X, y, df

    def train_resignation_model(self):
        X, y, _ = self.prepare_features()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        print(classification_report(y_test, y_pred))

        self.ml_models["resignation"] = model

    def predict_resignation_risk(self, employee_id):
        if "resignation" not in self.ml_models:
            print("Model not trained yet.")
            return None

        X, _, df = self.prepare_features()
        row = df[df["employee_id"] == employee_id]

        if row.empty:
            return None

        features = row[["salary", "avg_hours", "absent_ratio", "avg_rating",
                        "avg_training_score", "tenure_days", "budget"]]
        prob = self.ml_models["resignation"].predict_proba(features)[0][1]

        return {
            "employee_id": employee_id,
            "name": f"{row.iloc[0]['first_name']} {row.iloc[0]['last_name']}",
            "resignation_risk": prob,
            "impact_analysis": {
                "salary_loss": row.iloc[0]["salary"],
                "training_investment_loss": row.iloc[0]["avg_training_score"] * 1000,
                "department_budget_share": row.iloc[0]["salary"] / row.iloc[0]["budget"] if row.iloc[0]["budget"] > 0 else 0,
                "productivity_loss_hours": row.iloc[0]["avg_hours"] * 22
            }
        }

In [ ]:
try:
    print("=== INIT MODEL ===")
    hr_ml_model = HRPredictiveResignedModel()

    print("=== TRAINING ===")
    hr_ml_model.train_resignation_model()

    if "resignation" in hr_ml_model.ml_models:
        model = hr_ml_model.ml_models["resignation"]

        # Cek fitur yang dipakai model
        X, y, df = hr_ml_model.prepare_features()
        print("Model feature names:", X.columns.tolist())
        print("Prediction features:", X.columns.tolist())
        print("Feature match:", set(model.feature_names_in_) == set(X.columns))

        print("=== PREDICTION (contoh 10 pegawai) ===")
        sample_employees = df["employee_id"].sample(10, random_state=42).tolist()
        
        for emp_id in sample_employees:
            result = hr_ml_model.predict_resignation_risk(emp_id)
            
            if result:
                print(result)

        # Analisis agregat
        print("\n=== STATISTICS ===")
        preds = []
        
        for emp_id in df["employee_id"].tolist():
            result = hr_ml_model.predict_resignation_risk(emp_id)
            
            if result:
                preds.append({
                    "employee_id": emp_id,
                    "status": df[df["employee_id"] == emp_id]["status"].values[0],
                    "risk": result["resignation_risk"]
                })

        preds_df = pd.DataFrame(preds)
        print(f"Min risk: {preds_df['risk'].min():.2f}")
        print(f"Max risk: {preds_df['risk'].max():.2f}")
        print(f"Avg risk: {preds_df['risk'].mean():.2f}")
        print(f"Active employees avg risk: {preds_df[preds_df['status'].str.lower() == 'active']['risk'].mean():.2f}")
        print(f"Resigned employees avg risk: {preds_df[preds_df['status'].str.lower() == 'resigned']['risk'].mean():.2f}")

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()


In [ ]:
class HRPredictivePromotionModel(HRAnalyticsModel):
    def __init__(self):
        super().__init__()
        self.ml_models = {}

    def prepare_features(self):
        # Attendance features
        att_agg = self.attendance_df.groupby("employee_id").agg({
            "hours_worked": "mean",
            "status": lambda x: (x == "Absent").sum() / len(x)
        }).reset_index()
        att_agg.rename(columns={"hours_worked": "avg_hours", "status": "absent_ratio"}, inplace=True)

        # Performance features
        perf_agg = self.performance_df.groupby("employee_id")["rating"].mean().reset_index()
        perf_agg.rename(columns={"rating": "avg_rating"}, inplace=True)

        # Training features
        train_agg = self.training_df.groupby("employee_id")["score"].mean().reset_index()
        train_agg.rename(columns={"score": "avg_training_score"}, inplace=True)

        # Merge ke employees
        df = self.employees_df.merge(att_agg, on="employee_id", how="left") \
                              .merge(perf_agg, on="employee_id", how="left") \
                              .merge(train_agg, on="employee_id", how="left") \
                              .merge(self.departments_df, on="department_id", how="left")

        # Tenure
        df["hire_date"] = pd.to_datetime(df["hire_date"])
        df["tenure_days"] = (pd.Timestamp.now() - df["hire_date"]).dt.days

        # Label promotion (rule sederhana, bisa diubah sesuai data real)
        df["promoted"] = df["job_title"].str.lower().apply(
            lambda x: 1 if ("senior" in x or "manager" in x or "head" in x) else 0
        )

        # Bersihin NaN
        df.fillna(0, inplace=True)

        # Features
        X = df[["salary", "avg_hours", "absent_ratio", "avg_rating", 
                "avg_training_score", "tenure_days", "budget"]]
        y = df["promoted"]

        return X, y, df

    def train_promotion_model(self):
        X, y, _ = self.prepare_features()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        print(classification_report(y_test, y_pred))

        self.ml_models["promotion"] = model

    def predict_promotion_probability(self, employee_id):
        if "promotion" not in self.ml_models:
            print("Model not trained yet.")
            return None

        X, _, df = self.prepare_features()
        row = df[df["employee_id"] == employee_id]

        if row.empty:
            return None

        features = row[["salary", "avg_hours", "absent_ratio", "avg_rating",
                        "avg_training_score", "tenure_days", "budget"]]
        prob = self.ml_models["promotion"].predict_proba(features)[0][1]

        return {
            "employee_id": employee_id,
            "name": f"{row.iloc[0]['first_name']} {row.iloc[0]['last_name']}",
            "promotion_probability": prob,
            "factors": {
                "salary": row.iloc[0]["salary"],
                "avg_rating": row.iloc[0]["avg_rating"],
                "training_score": row.iloc[0]["avg_training_score"],
                "tenure_days": row.iloc[0]["tenure_days"],
                "absent_ratio": row.iloc[0]["absent_ratio"]
            }
        }

    def predict_all_promotions(self):
        if "promotion" not in self.ml_models:
            print("Model not trained yet.")
            return None

        X, _, df = self.prepare_features()
        probs = self.ml_models["promotion"].predict_proba(X)[:, 1]

        df_result = df.copy()
        df_result["promotion_probability"] = probs

        # Urutkan dari kandidat paling besar peluangnya
        df_result = df_result.sort_values(by="promotion_probability", ascending=False)

        return df_result[[
            "employee_id", "first_name", "last_name", "department_name", "job_title", 
            "salary", "avg_rating", "avg_training_score", "tenure_days", 
            "promotion_probability"
        ]]

In [ ]:
try:
    print("=== INIT MODEL ===")
    hr_promo_model = HRPredictivePromotionModel()

    print("=== TRAINING ===")
    hr_promo_model.train_promotion_model()

    if "promotion" in hr_promo_model.ml_models:
        print("\n=== PREDICT ONE EMPLOYEE ===")

        # ambil 1 random employee dari dataset
        _, _, df = hr_promo_model.prepare_features()
        sample_emp = df.sample(1, random_state=42).iloc[0]["employee_id"]

        result = hr_promo_model.predict_promotion_probability(sample_emp)
        if result:
            print("Prediction result:")
            print(result)

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()


In [ ]:
try:
    print("=== INIT MODEL ===")
    hr_promo_model = HRPredictivePromotionModel()

    print("=== TRAINING ===")
    hr_promo_model.train_promotion_model()

    if "promotion" in hr_promo_model.ml_models:
        print("\n=== PREDICT ALL EMPLOYEES ===")
        results = hr_promo_model.predict_all_promotion_probabilities()

        # tampilkan 5 hasil teratas untuk cek
        for r in results[:5]:
            print(r)

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
